# Advent of Code 2023: Day 19
https://adventofcode.com/2023/day/19


## Part 1


### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n\n')

### Create function to process the instruction part of the input into a dictionary

In [2]:
def processInstructions(data):
  rules = {}
  for rule in data:
    name = rule[:rule.find('{')]
    step = rule[rule.find('{')+1:rule.find('}')]
    steps = []
    for s in step.split(','):
      tmp = s.split(':')
      if len(tmp) == 2:
        o = tmp[0][:2]
        t = int(tmp[0][2:])
        steps.append([o, t, tmp[1]])
      else:
        steps.append(tmp)
    rules[name] = steps
  return rules

### Create function to process the item part of the input into a list of dictionaries

In [3]:
def processItems(data):
  items = []
  for item in data:
    i = {}
    values = item.split(',')
    for v in values:
      tmp = v[v.find('=')+1:]
      if v[0] == '{':
        i[v[1]] = int(v[v.find('=')+1:])
      elif tmp[-1] == '}':
        i[v[0]] = int(v[v.find('=')+1:-1])
      else:
        i[v[0]] = int(v[v.find('=')+1:])
    items.append(i)
  return items

### Create function to follow a single rule

In [4]:
def followRule(item, rule, rule_dict):
  steps = rule_dict[rule]
  for step in steps:
    if len(step) == 1:
      return step[0]
    else:
      comp = step[0][1]
      i_v = item[step[0][0]]
      if comp == '<' and i_v < step[1]:
        return step[2]
      elif comp == '>' and i_v > step[1]:
        return step[2]
  return 'Error'

### Create function to follow rules until an item is accepted or rejected

In [5]:
def followRules(item, rule, rule_dict):
  while rule != 'A' and rule != 'R':
    rule = followRule(item, rule, rule_dict)
  return rule

### Create function to go through the items and accept/reject them

In [6]:
def acceptReject(items, rule_dict):
  AR = []
  start_rule = 'in'
  for item in items:
    rule = followRules(item, start_rule, rule_dict)
    AR.append(rule)

  return AR

### Accept or Reject every item, sum up the values for the accepted items

In [7]:
rule_dict = processInstructions(data_list[0].split('\n'))
items = processItems(data_list[1].split('\n'))
AR = acceptReject(items, rule_dict)

total_value = 0
for i, item in enumerate(items):
  if AR[i] == 'A':
    total_value += sum(list(item.values()))
total_value

509597

## Part 2
Find how many distinct combinations would be accepted

### Create function to return all combinations of ranges that would cause an item to be Accepted.

In [8]:
from copy import deepcopy
def acceptRejectRange(ranges, rule, rule_dict):
  ranges_tmp = deepcopy(ranges)
  combos = []

  # If the rule is A, return the valid combination,
  # otherwise return an empty list
  if rule == 'A':
    return [ranges_tmp]
  elif rule == 'R':
    return combos

  # Get the current rule from the dictionary
  r = rule_dict[rule]

  # Check if all outcomes of the rule lead to A
  # or R. Same outomes as above
  all_A = True
  all_R = True
  for step in r:
    if step[-1] != 'A':
      all_A = False
      break
  for step in r:
    if step[-1] != 'R':
      all_R = False
      break
  if all_A:
    return [ranges_tmp]
  if all_R:
    return combos

  # Go through the steps of the rule
  for step in r:

    # it's the final step in a rule, go straight to the next rule
    if len(step) == 1:
      combos.append(acceptRejectRange(ranges_tmp, step[0], rule_dict))

    # Otherwise, modify the range corresponding to the current step so
    # that it passes the comparrison and go to the next rule. Afterwards,
    # instead modify it so that it fails the comparison (for the next iteration).
    else:
      char, comp = step[0]
      if comp == '<':
        p = ranges_tmp[char][1]
        ranges_tmp[char][1] = step[1]-1
        combos.append(acceptRejectRange(ranges_tmp, step[2], rule_dict))
        ranges_tmp[char][1] = p
        ranges_tmp[char][0] = step[1]
      elif comp == '>':
        p = ranges_tmp[char][0]
        ranges_tmp[char][0] = step[1]+1
        combos.append(acceptRejectRange(ranges_tmp, step[2], rule_dict))
        ranges_tmp[char][0] = p
        ranges_tmp[char][1] = step[1]
  return combos

### Create function to unpack a nested list

In [9]:
def unpackList(nested_list, inner_element):
  result = nested_list
  if inner_element == 'dict':
    while not all(isinstance(item, dict) for item in result):
      res = []
      i = 0
      for item in result:
        if isinstance(item, dict):
          res.append(item)
        else:
          res.extend(item)
        i += 1
      result = res
  return result

### Calculate the total number of combinations that would cause an item to be Accepted

In [10]:
ranges = {
    'x': [1,4000],
    'm': [1,4000],
    'a': [1,4000],
    's': [1,4000]
}

combos = acceptRejectRange(ranges, 'in', rule_dict)
combos = unpackList(combos, 'dict')
combos
total = 0
for item in combos:
  x = item['x']
  m = item['m']
  a = item['a']
  s = item['s']

  total += ((x[1]-x[0]+1) * (m[1]-m[0]+1) * (a[1]-a[0]+1) * (s[1]-s[0]+1))
total

143219569011526